In [13]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from Bio.Seq import Seq
#from Bio.Alphabet import generic_dna
from Bio.SeqRecord import SeqRecord
from Bio.SeqIO.QualityIO import FastqGeneralIterator

from itertools import chain
from typing import TextIO
import re

In [5]:
input_f = '../data/d14_plasmid_library_ratio_targeting_library.csv'
df = pd.read_csv(input_f,header=None)
df.columns =['guide','gene', 'gene_id', 'pos', 'raw ratio']

In [12]:
input_f = '../data/integrated_guide_feature_filtered_f24_mismatch3_all_features.csv'
df = pd.read_csv(input_f)

## guide free energy, Linearfold

In [14]:
# make linearfold input
standard_prefix = "CAAGTAAACCCCTACCAACTGGTCGGGGTTTGAAAC" # DR

num_examples = len(df['guide'].values)  
with open("../data/Linearfold_input/essential_guides_input.fasta", 'w') as file:
    for i in range(num_examples):
        file.write(">guide #" + str(i + 1) + "\n")
        whole_guide = standard_prefix + df['guide'].values[i]
        file.write(whole_guide.upper().replace('T','U') + "\n")
        

In [15]:
! cat ../data/Linearfold_input/essential_guides_input.fasta | ../Linearfold/./linearfold  > ../data/Linearfold_output/essential_guides_output.txt 

In [16]:
def parse_guide_linearfold_fasta_into_dict_contrafold(fname):
    fasta_file = open(fname)
    seq_dict = {}
    score_dict = {}

    def parse_one_example(fasta: TextIO):
        descr_line = fasta.readline()
        if not descr_line:
            return None, None, None
        guide_seq = fasta.readline().strip()[36:]
        linearfold_and_result = fasta.readline()
        match = re.match('([\\.|\\(|\\)]+) \((\-?[0-9]*\.[0-9]+)\)', linearfold_and_result)
        linseq, score = match.groups()
        score = float(score)

        assert '>' in descr_line

        return guide_seq, linseq, score

    while True:
        key, seq, score = parse_one_example(fasta_file)
        if key is None:
            break
        seq_dict[key] = seq
        score_dict[key] = score

    fasta_file.close()

    return seq_dict, score_dict

In [20]:
def parse_guide_linearfold_fasta_into_dict_contrafold_rna(fname):
    fasta_file = open(fname)
    seq_dict = {}
    score_dict = {}

    def parse_one_example(fasta: TextIO):
        descr_line = fasta.readline()
        if not descr_line:
            return None, None, None
        guide_seq = fasta.readline().strip()[36:]
        linearfold_and_result = fasta.readline()
        match = re.match('([\\.|\\(|\\)]+) \((\-?[0-9]*\.[0-9]+)\)', linearfold_and_result)
        linseq, score = match.groups()
        score = float(score)

        assert '>' in descr_line

        return guide_seq.replace('U','T'), linseq, score

    while True:
        key, seq, score = parse_one_example(fasta_file)
        if key is None:
            break
        seq_dict[key] = seq
        score_dict[key] = score

    fasta_file.close()

    return seq_dict, score_dict

In [21]:
# add guide mfe energy and direct repeat disruption feature
guide_linfold_c = '../data/Linearfold_output/essential_guides_output.txt'
#lin_seq_dict, lin_result_dict = parse_guide_linearfold_fasta_into_dict_contrafold(guide_linfold_c)
lin_seq_dict, lin_result_dict = parse_guide_linearfold_fasta_into_dict_contrafold_rna(guide_linfold_c)

linearfold_dr = [lin_seq_dict[guide][0:36] for guide in df['guide'].values]
ref_dr = '.....(((((((.(((....))).))))))).....'
dr_disr_num =0
for jj in range(num_examples):
    if linearfold_dr[jj] == ref_dr:
        linearfold_dr[jj] = 0
    else:
        linearfold_dr[jj] = 1
        dr_disr_num += 1
print('dr_disr_num:'+str(dr_disr_num))  

linearfold_vals = [lin_result_dict[guide] for guide in df['guide'].values]
for ii in range(num_examples):
    linearfold_vals[ii] = linearfold_vals[ii]-6.48
    
df['linearfold_vals_new']=linearfold_vals
df['linearfold_dr_flag_new']= linearfold_dr

dr_disr_num:12264


In [23]:
min(linearfold_vals)

0.0

In [24]:
df[['linearfold_vals','linearfold_vals_new']].corr(method='pearson')

,linearfold_vals,linearfold_vals_new
linearfold_vals,1.0,1.0
linearfold_vals_new,1.0,1.0


In [25]:
df[['linearfold_vals','linearfold_vals_new']].corr(method='spearman')

,linearfold_vals,linearfold_vals_new
linearfold_vals,1.0,1.0
linearfold_vals_new,1.0,1.0


## isoform target percent

In [3]:
refseq_gene_transcript = '../data/Refseq_all/Refseq_gene_info.txt'
df_info = pd.read_csv(refseq_gene_transcript,sep='\t')
#coding transcripts and genes
df_info_coding = df_info[df_info['name'].astype(str).str.startswith('NM')]
print(len(df_info_coding['name2'].values))

#make gene_dic of transcripts, coding
gene_transcript = {} #key:gene, value: transcript list
transcript_list =[]
for index, row in df_info_coding.iterrows():
    gene_n = row['name2']
    transcript_id = row['name']
    transcript_list.append(transcript_id)
    if gene_n in gene_transcript.keys():
        if transcript_id not in gene_transcript[gene_n]:
            gene_transcript[gene_n].append(transcript_id)
    else:
        gene_transcript[gene_n]=[transcript_id]
        
transcript_list = list(set(transcript_list))
print(len(transcript_list))

67542
60847


In [4]:
#get sequence for coding transcripts - not predicted
transcript_seq = {} #key: transcript, value: sequence
input_fasta = "../data/Refseq_all/Refseqall_gene_seq.fa" 
for record in SeqIO.parse(input_fasta, "fasta"):
    sequence = record.seq
    transcript_ID = record.id
    transcript_ID = transcript_ID.split('ncbiRefSeq_')[1]
    if (transcript_ID.startswith('NM')) and (transcript_ID not in transcript_seq.keys()):
        transcript_seq[transcript_ID]= sequence

In [6]:
# make guide dic for essential genes in our screen
gene_ess = ['RPS14', 'CDC5L', 'POLR2I', 'RPS7', 'XAB2', 'RPS19BP1', 'RPL23A', 'SUPT6H', 'PRPF31', 'U2AF1', 'PSMD7',
         'HSPE1', 'RPS13', 'PHB', 'RPS9', 'EIF5B', 'RPS6', 'RPS11', 'SUPT5H', 'SNRPD2', 'RPL37', 'RPSA', 'COPS6',
         'DDX51', 'EIF4A3', 'KARS1', 'RPL5', 'RPL32', 'SF3A1', 'RPS3A', 'SF3B3', 'POLR2D', 'RPS15A', 'RPL31', 'PRPF19',
         'SF3B2', 'RPS4X', 'CSE1L', 'RPL6', 'COPZ1', 'PSMB2', 'RPL7', 'PHB2', 'ARCN1', 'RPA2', 'NUP98', 'RPS3', 'EEF2',
         'USP39', 'PSMD1', 'NUP93', 'AQR', 'RPL34', 'PSMA1', 'RPS27A']
print(len(gene_ess))
guide_dic ={}

for gene in gene_ess:
    gene_guide_dic = {} #gene-specific guide dic
    trans_list_all = gene_transcript[gene] #may include non-coding transcripts
    trans_list = [] #only include coding transcripts that start with 'NM'
    for tt in trans_list_all:
        if tt.startswith('NM'):
            trans_list.append(tt)
    trans_num = len(trans_list) #isoform total number
    for trans_id in trans_list:
        if trans_id not in transcript_seq.keys():
            print('version different')
            id_short = trans_id.split('.')[0] #use one without version id
            id_versions = 0 # how many isoform with different versions for this short id
            for t in transcript_seq.keys():
                if id_short in t: #transcript id matches, version different
                    id_versions += 1
                    #trans_id = t
                    #design the guides
                    trans_seq = transcript_seq[t]
                    for i in range(0,len(trans_seq)-29):
                        target = trans_seq[i:i+30]
                        spacer = target.reverse_complement()
                        if str(spacer) not in gene_guide_dic.keys(): #first occur
                            gene_guide_dic[str(spacer)]=[str(spacer),[t],[i],1,1] 
                                                    #spacer, transcript_list,pos_list,target_trans_num, target_pos_num_for_one_transcript
                        else:
                            if t not in gene_guide_dic[str(spacer)][1]:
                                gene_guide_dic[str(spacer)][1].append(t)
                                gene_guide_dic[str(spacer)][2].append(i)
                                if id_versions == 1: #only add count if it's the first isoform for this transcript
                                    gene_guide_dic[str(spacer)][3] += 1 #target transcript number
                            else: #guide targeting the same transcript
                                gene_guide_dic[str(spacer)][4] += 1
                                
            
        else:
            #design the guides
            trans_seq = transcript_seq[trans_id]
            for i in range(0,len(trans_seq)-29):
                target = trans_seq[i:i+30]
                spacer = target.reverse_complement()
                if str(spacer) not in gene_guide_dic.keys(): #first occur
                    gene_guide_dic[str(spacer)]=[str(spacer),[trans_id],[i],1,1]
                                        #spacer, transcript_list,pos_list,target_trans_num, target_pos_num_for_one_transcript
                else:
                    if trans_id not in gene_guide_dic[str(spacer)][1]:
                        gene_guide_dic[str(spacer)][1].append(trans_id)
                        gene_guide_dic[str(spacer)][2].append(i)
                        gene_guide_dic[str(spacer)][3] += 1 #target transcript number
                    else: #guide targeting the same transcript
                        gene_guide_dic[str(spacer)][4] += 1
                        
    for guide in gene_guide_dic.keys():
        if guide not in guide_dic.keys():
            target_percent = float(gene_guide_dic[guide][3])/trans_num
            guide_dic[guide]= gene_guide_dic[guide]+[target_percent]
        else: #guide target other genes
            guide_dic.pop(guide) #remove the guide with off targets
            

guide_iso = pd.DataFrame.from_dict(guide_dic, orient='index',columns=['guide','transcript_list','target_pos_list','target transcript number','target pos num','target_transcript_percent'])
guide_iso.head()

55


,guide,transcript_list,target_pos_list,target transcript number,target pos num,target_transcript_percent
TCGTCTCCAGACTCCACACCGGAAAGAGAG,TCGTCTCCAGACTCCACACCGGAAAGAGAG,"[NM_001025071.2, NM_001025070.2, NM_005617.4]","[0, 0, 0]",3,1,1.0
GTCGTCTCCAGACTCCACACCGGAAAGAGA,GTCGTCTCCAGACTCCACACCGGAAAGAGA,"[NM_001025071.2, NM_001025070.2, NM_005617.4]","[1, 1, 1]",3,1,1.0
CGTCGTCTCCAGACTCCACACCGGAAAGAG,CGTCGTCTCCAGACTCCACACCGGAAAGAG,"[NM_001025071.2, NM_001025070.2, NM_005617.4]","[2, 2, 2]",3,1,1.0
ACGTCGTCTCCAGACTCCACACCGGAAAGA,ACGTCGTCTCCAGACTCCACACCGGAAAGA,"[NM_001025071.2, NM_001025070.2, NM_005617.4]","[3, 3, 3]",3,1,1.0
CACGTCGTCTCCAGACTCCACACCGGAAAG,CACGTCGTCTCCAGACTCCACACCGGAAAG,"[NM_001025071.2, NM_001025070.2, NM_005617.4]","[4, 4, 4]",3,1,1.0


In [ ]:
with open('essential_guide_target_percent.csv','w') as outf:
    writer = csv.writer(outf)
    writer.writerow(['guide','transcript_list','target_pos_list','target transcript number','target pos num','target_transcript_percent'])
    for g in guide_dic.keys():
        writer.writerow(guide_dic[g])

## position flags and floats

In [ ]:
#create dictionary of transcripts
transcript_dictionary = {}
gene_len = 'lengths.csv'
gene_cds = "CDS.csv"

#CDS position
with open(gene_cds, 'r') as f:
	for row in csv.reader(f.read().splitlines()):
		transcript = str(row[1])
		transcript_dictionary[transcript] = row[2:4] #CDS position
        
#gene length
with open(gene_len, 'r') as f:
	for row in csv.reader(f.read().splitlines()):
		transcript = str(row[1])
		transcript_dictionary[transcript] = transcript_dictionary[transcript]+[row[-1]] #gene length
        
print(len(transcript_dictionary))

In [ ]:
#CDS, 5'UTR, 3'UTR pos floats
output_dictionary = {}
#is_CDS = []
#is_5UTR = []
#is_3UTR = []
CDS_position = []
UTR5_position = []
UTR3_position = []


#CDS, 5'UTR, 3'UTR
for guide, attributes in guide_dictionary.items():
	transcript = attributes[2] 
	position = int(attributes[-20])
#	print(position)
#	break
	CDS_start = int(transcript_dictionary[transcript][0])
	CDS_end = int(transcript_dictionary[transcript][1])
	gene_length = int(transcript_dictionary[transcript][2]) 
	if position > CDS_start: #if the guide starts after the start of the CDS
		if position < CDS_end : # and before the end of the CDS
#			is_CDS.append(guide_dictionary[guide] + [1])
			rel_position = float(position - CDS_start) / float(CDS_end - CDS_start)
			output_dictionary[guide]= guide_dictionary[guide] + [0,rel_position,0]
			CDS_position.append(guide_dictionary[guide] + [rel_position])
#			is_3UTR.append(guide_dictionary[guide] + [0])
#			is_5UTR.append(guide_dictionary[guide] + [0])
			UTR3_position.append(guide_dictionary[guide] + [0])
			UTR5_position.append(guide_dictionary[guide] + [0])
		else:
#			is_3UTR.append(guide_dictionary[guide] + [1])
#			is_5UTR.append(guide_dictionary[guide] + [0])
#			is_CDS.append(guide_dictionary[guide] + [0])
			rel_position = float(position - CDS_end) / float(gene_length-CDS_end)
			output_dictionary[guide]= guide_dictionary[guide] + [0,0,rel_position]
			UTR3_position.append(guide_dictionary[guide] + [rel_position]) 
			CDS_position.append(guide_dictionary[guide] + [0])
			UTR5_position.append(guide_dictionary[guide] + [0])
	else: 
#		is_5UTR.append(guide_dictionary[guide] + [1])
#		is_CDS.append(guide_dictionary[guide] + [0])
#		is_3UTR.append(guide_dictionary[guide] + [0])
		CDS_position.append(guide_dictionary[guide] + [0])
		UTR3_position.append(guide_dictionary[guide] + [0])
		rel_position = float(position)/float(CDS_start)
		UTR5_position.append(guide_dictionary[guide] + [rel_position])
		output_dictionary[guide]= guide_dictionary[guide] + [rel_position,0,0]

## flanking sequences

In [ ]:
guides_pos = {}

input_fasta = "essential_transcripts.fasta"

for record in SeqIO.parse(input_fasta, "fasta"):
	sequence = record.seq
	transcript_ID = record.id
	title = str(record.description)

	#find the gene symbol
	a = title.find("(")
	b = title.find(")")
#	gene_symbol = title.split( )[1]
	gene_symbol = title[a+1:b]

	#design the guides
	for i in range(0,len(record.seq)-29):
		target = sequence[i:i+30]
		spacer = target.reverse_complement()
		if (i != 0) and (i != len(record.seq)-30):
			left_seq_5 = sequence[max(0,(i-5)):i]
			right_seq_5 = sequence[i+30:min((i+35),len(record.seq))]
			left_seq_10 = sequence[max(0,(i-10)):i]
			right_seq_10 = sequence[i+30:min((i+40),len(record.seq))]
			left_seq_15 = sequence[max(0,(i-15)):i]
			right_seq_15 = sequence[i+30:min((i+45),len(record.seq))]
			left_seq_20 = sequence[max(0,(i-20)):i]
			right_seq_20 = sequence[i+30:min((i+50),len(record.seq))]
			left_seq_100 = sequence[max(0,(i-100)):i]
			right_seq_100 = sequence[i+30:min((i+130),len(record.seq))]
		elif i == 0:
			left_seq_5 = ''
			right_seq_5 = sequence[i+30:min((i+35),len(record.seq))]
			left_seq_10 = ''
			right_seq_10 = sequence[i+30:min((i+40),len(record.seq))]
			left_seq_15 = ''
			right_seq_15 = sequence[i+30:min((i+45),len(record.seq))]
			left_seq_20 = ''
			right_seq_20 = sequence[i+30:min((i+50),len(record.seq))]
			left_seq_100 = ''
			right_seq_100 = sequence[i+30:min((i+130),len(record.seq))]
		elif i == len(record.seq)-30:  
			left_seq_5 = sequence[max(0,(i-5)):i]
			right_seq_5 = ''
			left_seq_10 = sequence[max(0,(i-10)):i]
			right_seq_10 = ''
			left_seq_15 = sequence[max(0,(i-15)):i]
			right_seq_15 = ''
			left_seq_20 = sequence[max(0,(i-20)):i]
			right_seq_20 = ''
			left_seq_100 = sequence[max(0,(i-100)):i]
			right_seq_100 = ''
		nearby_seq_all_5 = left_seq_5 + target + right_seq_5 #target seq+flanks
		nearby_seq_all_10 = left_seq_10 + target + right_seq_10
		nearby_seq_all_15 = left_seq_15 + target + right_seq_15 
		nearby_seq_all_20 = left_seq_20 + target + right_seq_20 
		nearby_seq_all_100 = left_seq_100 + target + right_seq_100 #target seq+flanks
#		nearby_seq_all_rv  = nearby_seq_all.reverse_complement()
#		guide_loc_flag = np.zeros(len(nearby_seq_all))
#		guide_loc = nearby_seq_all_rv.find(spacer)
#		guide_loc_flag[guide_loc:guide_loc+30]=1 # 1 means where the guide is in this flag vector
		if str(spacer) not in guides_pos:
#			guides_pos[spacer]= [gene_symbol,transcript_ID,i,str(left_seq),str(right_seq),str(nearby_seq_all),str(nearby_seq_all_rv),guide_loc_flag]
			guides_pos[spacer]= [gene_symbol,transcript_ID,i,str(target),str(left_seq_5),str(right_seq_5),str(nearby_seq_all_5),str(nearby_seq_all_10),str(nearby_seq_all_15),str(nearby_seq_all_20),str(nearby_seq_all_100)]

In [ ]:
#target with flank 15 nt
#native
nearby_seq_all = df['nearby_seq_all_15'].values
with open("Linearfold_input/essential_target_flank15_input.fasta", 'w') as file:
    for i in range(num_examples):
        file.write(">guide #" + str(i + 1) + "\n")
        file.write(nearby_seq_all[i] + "\n")
        
# with constraints
guide_rv = df['target_seq'].values
with open("Linearfold_input/essential_target_constraints_flank15_input.fasta", 'w') as file:
    for i in range(num_examples):
        file.write(">guide #" + str(i + 1) + "\n")
        file.write(nearby_seq_all[i] + "\n")
        target_index = nearby_seq_all[i].find(guide_rv[i])
        constraints = "?"*target_index + "."*30 + "?"*(len(nearby_seq_all[i])-30-target_index)
        file.write(constraints + "\n")